# K-fold Cross Validation I - Predicting Burnout

In [1]:
#Start with importing necessary libraries 
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.api import OLS

In [2]:
#Reading my data saved as a CSV file
df = pd.read_csv('predictingburnout.csv')

In [3]:
#Check how the dataframe looks
df.head()

,nqgender,nqbirthdate,nqfaculty,nqstyear,Ethnicity,Living_situation,SenseBelonging_A,SenseBelonging_B,SenseBelonging_C,SenseBelonging_D,...,NormExhaustion,NormDistance,NormCompetence,high_exhaus,high_distance,low_competence,BurnoutYES,MEANComp2,NormComp2,high_Exhaus_CBS
0,M,03-09-00,Bewegen en Educatie,1,3,2,4,3,2,4,...,2,3,3,0,0,0,0,4.6,3,0
1,M,11/29/1996,Bewegen en Educatie,1,3,1,4,3,2,4,...,2,1,2,0,0,1,0,3.8,3,0
2,M,07-10-00,Bewegen en Educatie,1,3,2,5,3,1,5,...,4,3,3,1,0,0,0,4.0,3,0
3,M,6/23/1999,Bewegen en Educatie,1,3,2,5,5,1,5,...,2,1,4,0,0,0,0,5.6,4,0
4,M,2/27/1999,Bewegen en Educatie,1,1,2,4,4,1,4,...,4,3,2,1,0,1,1,3.6,2,0


In [4]:
#Check if number of data rows are correct
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3141 entries, 0 to 3140
Columns: 116 entries, nqgender to high_Exhaus_CBS
dtypes: float64(13), int64(100), object(3)
memory usage: 2.8+ MB


In [5]:
#There are so many columns in the dataset.
print(df.columns.tolist())

['nqgender', 'nqbirthdate', 'nqfaculty', 'nqstyear', 'Ethnicity', 'Living_situation', 'SenseBelonging_A', 'SenseBelonging_B', 'SenseBelonging_C', 'SenseBelonging_D', 'SenseBelonging_E', 'SenseBelonging_F', 'ExperiencedStress_A', 'UBOS_A_A', 'UBOS_A_B', 'UBOS_A_C', 'UBOS_A_D', 'UBOS_A_E', 'UBOS_A_F', 'UBOS_A_G', 'UBOS_A_H', 'UBOS_A_I', 'UBOS_A_J', 'UBOS_A_K', 'UBOS_A_L', 'UBOS_A_M', 'UBOS_A_N', 'UBOS_A_O', 'Loneliness_A', 'Loneliness_B', 'Loneliness_C', 'Loneliness_D', 'Loneliness_E', 'Loneliness_F', 'Loneliness_G', 'Loneliness_H', 'Loneliness_I', 'Loneliness_J', 'Loneliness_K', 'PerformancePressure_A', 'PerformancePressure_B', 'PerformancePressure_C', 'PerformancePressure_D', 'PerformancePressure_E', 'PerformancePressure_F', 'PerformancePressure_G', 'PerformancePressure_H', 'PerformancePressure_I', 'PerformancePressure_J', 'PerformancePressure_K', 'PerformancePressure_L', 'IncreaseInPressure', 'LON1', 'LON4', 'LON7', 'LON8', 'LON11', 'LON2', 'LON3', 'LON5', 'LON6', 'LON9', 'LON10', 'LO

In [6]:
# Create a new df with only the columns used in the original study
toselect = ['Age', 'Gender','Living_situation','StudyYear2','StudyYear3','StudyYear4','StudyYear5','SoB_total','LON_total','PerformancePressure_A','MeanDist','MeanComp','MeanUitp']
usevar = df[toselect]

print(usevar)

            Age  Gender  Living_situation  StudyYear2  StudyYear3  StudyYear4  \
0     17.815195       2                 2           0           0           0   
1     21.089665       2                 1           0           0           0   
2     17.478439       2                 2           0           0           0   
3     18.527036       2                 2           0           0           0   
4     18.844627       2                 2           0           0           0   
...         ...     ...               ...         ...         ...         ...   
3136  24.021903       1                 2           0           0           0   
3137  21.596167       1                 2           0           0           0   
3138  28.131417       1                 1           0           0           0   
3139  23.466119       1                 1           0           0           0   
3140  23.709788       1                 1           0           0           0   

      StudyYear5  SoB_total

In [7]:
usevar.describe()

,Age,Gender,Living_situation,StudyYear2,StudyYear3,StudyYear4,StudyYear5,SoB_total,LON_total,PerformancePressure_A,MeanDist,MeanComp,MeanUitp
count,3141.000000,3141.000000,3141.000000,3141.000000,3141.000000,3141.000000,3141.000000,3141.000000,3141.000000,3141.000000,3141.000000,3141.000000,3141.000000
mean,21.849311,1.399554,1.706145,0.212989,0.217765,0.184336,0.103470,3.808341,3.091054,2.851003,1.570201,3.660883,2.592932
std,3.450495,0.489885,0.455599,0.409485,0.412793,0.387820,0.304621,0.594668,3.339970,0.878307,1.395999,0.834422,1.335116
min,17.015743,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.605749,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.500000,0.000000,2.000000,0.500000,3.166667,1.600000
50%,21.245722,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,3.833333,2.000000,3.000000,1.250000,3.666667,2.400000
75%,23.321013,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,4.166667,5.000000,3.000000,2.250000,4.166667,3.400000
max,59.852156,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,5.000000,11.000000,4.000000,6.000000,6.000000,6.000000


In [8]:
#Drop outcome variables
X = usevar.drop(columns=['MeanDist', 'MeanComp', 'MeanUitp'])

# Add intercept--SM won't do this for us
X_ = sm.add_constant(X)

X_.head()

,const,Age,Gender,Living_situation,StudyYear2,StudyYear3,StudyYear4,StudyYear5,SoB_total,LON_total,PerformancePressure_A
0,1.0,17.815195,2,2,0,0,0,0,3.833333,0,2
1,1.0,21.089665,2,1,0,0,0,0,3.833333,9,4
2,1.0,17.478439,2,2,0,0,0,0,4.666667,3,3
3,1.0,18.527036,2,2,0,0,0,0,4.333333,1,3
4,1.0,18.844627,2,2,0,0,0,0,3.666667,5,3


In [9]:
#Separate your outcome variables from the predictors
y1 = usevar['MeanDist'].to_frame()
y2 = usevar['MeanComp'].to_frame()
y3 = usevar['MeanUitp'].to_frame()

In [10]:
#Depersonalization as outcome
model = sm.OLS(y1, X_)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               MeanDist   R-squared:                       0.365
Model:                            OLS   Adj. R-squared:                  0.363
Method:                 Least Squares   F-statistic:                     179.8
Date:                Sat, 03 Jun 2023   Prob (F-statistic):          1.52e-299
Time:                        12:52:15   Log-Likelihood:                -4791.5
No. Observations:                3141   AIC:                             9605.
Df Residuals:                    3130   BIC:                             9672.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     5.27

In [27]:
#Import Kfold cross-validation tools
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

#Indicate how many folds you want
kf = KFold(n_splits=5, shuffle=True)  # 'k' represents the number of folds

model = LinearRegression()

#Aplly it to your model
scores = cross_val_score(model, X_, y1, cv=kf, scoring='r2')
print('Cross-Validation Scores:', scores)
print('Mean R^2:', np.mean(scores))

Cross-Validation Scores: [0.33138255 0.33176722 0.3285802  0.40069545 0.39703224]
Mean R^2: 0.35789153360181575


In [13]:
#Personal accomplishment as outcome
model = sm.OLS(y2, X_)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               MeanComp   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     36.11
Date:                Sat, 03 Jun 2023   Prob (F-statistic):           1.86e-67
Time:                        12:52:49   Log-Likelihood:                -3716.4
No. Observations:                3141   AIC:                             7455.
Df Residuals:                    3130   BIC:                             7521.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.31

In [14]:
#Emotional Exhaustion as outcome
model = sm.OLS(y3, X_)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               MeanUitp   R-squared:                       0.289
Model:                            OLS   Adj. R-squared:                  0.286
Method:                 Least Squares   F-statistic:                     127.0
Date:                Sat, 03 Jun 2023   Prob (F-statistic):          6.51e-223
Time:                        12:52:53   Log-Likelihood:                -4829.4
No. Observations:                3141   AIC:                             9681.
Df Residuals:                    3130   BIC:                             9747.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     4.48

**Cross-validation resulted the same R^2 values. There is no drop in the prediction capacity. If the sample size was 100, how would the R2 value and coefficients change? Although 100 participants are not a realistic scenario for such a study but let's experiment.**

In [19]:
#Sample 100 people out of  3141.  
rs = usevar.sample(n=100)

In [21]:
M = rs.drop(columns=['MeanDist', 'MeanComp', 'MeanUitp'])
y = rs['MeanUitp'].to_frame() #Emotional Exhaustion as outcome

# Add intercept--SM won't do this for us
M_ = sm.add_constant(M)

In [63]:
model = sm.OLS(y, M_)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               MeanUitp   R-squared:                       0.381
Model:                            OLS   Adj. R-squared:                  0.360
Method:                 Least Squares   F-statistic:                     17.80
Date:                Fri, 02 Jun 2023   Prob (F-statistic):           3.25e-25
Time:                        14:58:43   Log-Likelihood:                -452.60
No. Observations:                 300   AIC:                             927.2
Df Residuals:                     289   BIC:                             967.9
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     5.25

In [33]:
#Aplly cross-validation to your new model (and notice that the Mean R^2 is variable when you run this code several times)

kf = KFold(n_splits=10, shuffle=True)  # 'k' represents the number of folds

model = LinearRegression()

scores = cross_val_score(model, M_, y, cv=kf, scoring='r2')
print('Cross-Validation Scores:', scores)
print('Mean R^2:', np.mean(scores))
print(np.std(scores))

Cross-Validation Scores: [ 0.66076364 -2.68334815 -0.26613985  0.30880153  0.25735947  0.05750366
  0.05907844  0.21946391  0.17384167  0.69549841]
Mean R^2: -0.0517177270827882
0.9171445711657181


In [32]:
#ADD REPETITION
# Define the number of repetitions
repetitions = 500

#Store the R-squared values for each repetition to an np array
rsquared_values = np.zeros(repetitions)

#Apply cross-validation 200 times with a for loop
for i in range(repetitions):
    scores = cross_val_score(model, M_, y, cv=kf, scoring='r2')
    
    # Record the mean R-squared value for this repetition
    rsquared_values[i] = np.mean(scores)

# Calculate the mean and standard deviation of the R-squared means
std_dev = np.std(rsquared_values)
mean_rsquared = np.mean(rsquared_values)

# Print the standard deviation of the R-squared means
print("Mean of R-squared Means:", mean_rsquared)
print("Standard Deviation of R-squared Means:", std_dev)

Mean of R-squared Means: 0.20375204839525685
Standard Deviation of R-squared Means: 0.11118408698004574
